In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

In [11]:
 data=pd.read_csv('../input/digit-recognizer/train.csv')


In [12]:
data=np.array(data)
m,n=data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

In [13]:
data_test=data[0:1000].T  ##transpose
Y_test=data_test[0]
X_test=data_test[1:n]

data_train=data[1000:m].T
Y_train=data_train[0]
X_train=data_train[1:n]
X_train = X_train / 255.
X_train.shape

In [14]:
## generating weights and biases randomly for two  layers(hidden and output)
def init_params():
    w1=np.random.rand(10,784)-0.5
    b1=np.random.rand(10,1) -0.5 #m
    w2=np.random.rand(10,10)-0.5
    b2=np.random.rand(10,1)-0.5
    
    return w1,b1,w2,b2


In [ ]:
def ReLU(x):
    return np.maximum(0,x)


def softmax(x):
    return np.exp(x)/sum(np.exp(x))
    

def forward_propagation(X,w1,b1,w2,b2):
    z1=w1.dot(X)+b1
    A1=ReLU(z1)
    z2=w2.dot(A1)+b2
    A2=softmax(z2)
    
    return z1,A1,z2,A2

#w1,b1,w2,b2=init_params()
#w1.shape
#z1,A1,z2,A2=forward_propagation(X_train,w1,b1,w2,b2)



def one_hot_encode(Y):
    one_hot_Y=np.zeros((Y.size,10))
    one_hot_Y[np.arange(Y.size),Y]=1
    one_hot_Y= one_hot_Y.T
    return  one_hot_Y

def derv_ReLU(z):
    return z>0


def back_propagation(X,Y,z1,A1,z2,A2,w2):
        m=Y.size
        one_hot_Y= one_hot_encode(Y)
        d2=A2- one_hot_Y
        dw2=1/m*d2.dot(A1.T)
        db2=1/m*np.sum(d2)
        d1=w2.T.dot(d2)+ derv_ReLU(z1) 
        dw1=1/m*d1.dot(X.T)
        db1=1/m*np.sum(d1) 
        
        return dw1, db1, dw2, db2

def update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha):
    w1=w1- alpha*dw1
    b1=b1-alpha*db1
    w2=w2- alpha*dw2
    b2=b2-alpha*db2
    return w1,b1,w2,b2
    

In [15]:
def get_predictions(A2):
    return np.argmax(A2,0)

def accuracy(predictions, Y):
    print(Y, predictions)
    return np.sum(predictions==Y)/Y.size


##it will return updated w1,b1,w2,b2 that are good for digit recognition
def gradient_descent(X,Y, iterations,alpha):
        w1,b1,w2,b2 = init_params()
       ## print(w1.shape)
        for i in range(iterations):
            z1,A1,z2,A2=forward_propagation(X,w1,b1,w2,b2) 
            dw1, db1, dw2, db2=back_propagation(X,Y,z1,A1,z2,A2,w2)
            w1,b1,w2,b2=update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha)
            if i%10==0:
                print("Iteration:",i)
                print("Accuracy: ",accuracy(get_predictions(A2),Y))
            
            
        return w1,b1,w2,b2

In [16]:
w1,b1,w2,b2=gradient_descent(X_train,Y_train, 500,0.1)